In [1]:
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
import time
import matplotlib.pyplot as plt
from keras.api.layers import *
from keras.api.models import Model, Sequential
from keras.api.regularizers import l2
from keras.api.optimizers import Adam
import keras.api.backend as K
import keras

import random
import numpy as np
import pickle
import json
import os

from module_design import model_structure

root_path = os.getcwd()
print(root_path)

board_size = 15
last_n_feature=1
l2_const = 1e-4
batch_size=512
epochs=50
init_lr=1e-4
is_load_model=False
load_model_dir=None# xx_xx_xxxxxx

sample_size=1900 # number of games choose from data
take_last_n_move=10 # number of last steps used in each game


2024-07-07 00:33:59.356248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-07 00:33:59.366390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-07 00:33:59.366409: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-07 00:33:59.373391: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-07 00:33:59.770060: W tensorflow/co

/home/james/projects/pythonProjects/TF7_final_project/model_pre_train


In [4]:
data_dir = os.path.join(root_path,f'../data_transform/tmp/training_data_v{last_n_feature}')
data_files = os.listdir(data_dir)
data = []
for file in data_files:
    print(f"loading file {file}...")
    file = os.path.join(data_dir,file)
    with open(file, 'rb') as f:
        data += pickle.load(f)
    print(f"file {file} loaded")
print(f"total of {len(data)} game(s) loaded")

status = []
probs = []
wins = []
result = []
moves = []
if sample_size > len(data): 
    sample_size=len(data)
data=random.sample(data,sample_size)
print(f"total of {len(data)} game(s) used")

for game in data:
    for move_n in game:
        moves.append(move_n)
random.shuffle(moves)
# print(moves[0])
for i in moves:
    status.append(i[0])
    probs.append(i[1])
    wins.append(i[2])

train_size = int(0.8*len(status))
print("total status:{}".format(len(status)))

x_train = status[:train_size]
x_test = status[train_size:]

y_train_probs = probs[:train_size]
y_test_probs = probs[train_size:]

y_train_wins = wins[:train_size]
y_test_wins = wins[train_size:]

x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

y_train_probs = np.asarray(y_train_probs)
y_train_probs = np.reshape(y_train_probs, (y_train_probs.shape[0], -1))
y_train_wins = np.asarray(y_train_wins)

y_test_probs = np.asarray(y_test_probs)
y_test_probs = np.reshape(y_test_probs, (y_test_probs.shape[0], -1))
y_test_wins = np.asarray(y_test_wins)


loading file trans_from_playok...
file /home/james/projects/pythonProjects/TF7_final_project/model_pre_train/../data_transform/tmp/training_data_v1/trans_from_playok loaded
total of 1925 game(s) loaded
total of 1900 game(s) used
total status:9500


In [6]:
if(is_load_model):
    model=load_model(os.path.join(root_path,'model_record',load_model_dir,'model.h5'))
else:
    model=model_structure(board_size,last_n_feature,l2_const)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=init_lr,
    decay_steps=10000,
    decay_rate=0.9)

optimizer = Adam(
    learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999)

model.compile(
    optimizer=optimizer,
    loss={'policy_net': 'categorical_crossentropy', 'value_net': 'mean_squared_error'},
    metrics={
        'policy_net': ['accuracy'], 
        'value_net': ['mae', 'mean_squared_error']
    })

# print(model.summary())

try:
    start_time=time.process_time()
    history = model.fit(x_train,
                    [y_train_probs, y_train_wins],
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.2,
                    validation_data=(x_test, [y_test_probs, y_test_wins]))
    process_time=time.process_time()-start_time
except KeyboardInterrupt as e:
    print(e)

result=history

Epoch 1/50


I0000 00:00:1720283768.981987   20301 service.cc:145] XLA service 0x7fb800003da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1720283768.982006   20301 service.cc:153]   StreamExecutor device (0): NVIDIA Graphics Device, Compute Capability 8.9
2024-07-07 00:36:10.459944: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-07 00:36:15.615135: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1720283780.753581   20869 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_58320', 412 bytes spill stores, 412 bytes spill loads

I0000 00:00:1720283780.819296   20881 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_58563', 244 bytes spill stores, 244 bytes spill loads

I0000

I0000 00:00:1720283824.040388   20301 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


NameError: name 'history' is not defined

In [ ]:
print(result.history.keys())

policy_net_accuracy = history.history['policy_net_accuracy']
val_policy_net_accuracy = history.history['val_policy_net_accuracy']
value_net_mae = history.history['value_net_mae']
val_value_net_mae = history.history['val_value_net_mae']
value_net_mse = history.history['value_net_mean_squared_error']
val_value_net_mse = history.history['val_value_net_mean_squared_error']

# Plotting Policy Net Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(2, 2, 1)
plt.plot(policy_net_accuracy, label='Policy Net Training Accuracy')
plt.plot(val_policy_net_accuracy, label='Policy Net Validation Accuracy')
plt.title('Policy Net Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting Value Net MAE
plt.subplot(2, 2, 2)
plt.plot(value_net_mae, label='Value Net Training MAE')
plt.plot(val_value_net_mae, label='Value Net Validation MAE')
plt.title('Value Net Training and Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()

# Plotting Value Net MSE
plt.subplot(2, 2, 3)
plt.plot(value_net_mse, label='Value Net Training MSE')
plt.plot(val_value_net_mse, label='Value Net Validation MSE')
plt.title('Value Net Training and Validation MSE')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()

plt.tight_layout()

t = time.localtime()
folder_name = f"{t.tm_mon:0>2}_{t.tm_mday:0>2}_{t.tm_hour:0>2}{t.tm_min:0>2}{t.tm_sec:0>2}"
new_folder_dir=os.path.join(root_path,'model_pre_train/model_record',folder_name)
os.makedirs(new_folder_dir, exist_ok=True)

plt.savefig(os.path.join(new_folder_dir,'history.png'))
plt.show()

keras.saving.save_model(model, os.path.join(new_folder_dir,'model.keras'))

def convert_seconds(seconds):
    seconds=int(seconds)
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours:0>3}h {minutes:0>2}m {seconds:0>2}s"

score = model.evaluate(
    x_test, [y_test_probs, y_test_wins], verbose=1)

with open(os.path.join(new_folder_dir,'result.txt'), 'w') as f:
    f.write(f"board size = {board_size}\n")
    f.write(f"load model = {is_load_model}\n")
    f.write(f"load model dir = {load_model_dir}\n")
    f.write(f"epochs = {epochs}\n")
    f.write(f"batch_size = {batch_size}\n")
    f.write(f"sample_size = {sample_size}\n")
    f.write(f"take_last_n_move = {take_last_n_move}\n")
    f.write(f"last_n_feature = {last_n_feature}\n")
    f.write(f"init lr = {init_lr}\n")
    f.write(f"l2 const = {l2_const}\n")
    f.write(f"loss = {score[0]}\n")
    f.write(f"policy_net_accuracy = {score[1]}\n")
    f.write(f"value_net_mae = {score[2]}\n")
    f.write(f"value_net_mean_squared_error = {score[3]}\n")
    f.write(f"process time = {convert_seconds(process_time)}\n")